

## Import modules and set Yellowfin Base URL



In [1]:
import requests 
import time
import json

YF_BASE_URL = "<YF_BASE_URL>" # Could be something like: "http://localhost:9696"

## Make Sure YF Instance is up and running!

In [3]:
r = requests.get(url = f"{YF_BASE_URL}/logonCheck.i4", verify=False)
assert r.status_code == 401, "Hmm, I can't seem to access the instance. Is it running?"
print(f"{YF_BASE_URL} is up and running! 💪")

http://localhost:9696/api is up and running! 💪


## Step 1 - Obtain Refresh Token

In [ ]:
millis = int(round(time.time() * 1000))

headers = {
    "Authorization": f"YELLOWFIN ts={millis}, nonce=random",
    "Content-Type": "application/json",
    "Accept": "application/vnd.yellowfin.api-v1+json",
}

payload = json.dumps(
    {
        "userName": "", # Change to ad admin account in your yellowfin instance
        "password": "",
        "clientOrgRef": "YFDefault",
    }
)

REFRESH_TOKEN_URL = f"{YF_BASE_URL}/api/refresh-tokens"

r = requests.request("POST", url=REFRESH_TOKEN_URL, headers=headers, data=payload)

r_dict = json.loads(r.text)
refresh_token = r_dict["securityToken"]
print("Refresh Token Obtained:")
print(refresh_token + "\n")

## Step 2 - Use Refresh Token to Obtain Access Token

In [ ]:
ACCESS_TOKEN_URL = f'{YF_BASE_URL}/api/access-tokens'

millis = int(round(time.time() * 1000))
headers = {
    "Accept": "application/vnd.yellowfin.api-v1+json",
    "Authorization": f"YELLOWFIN ts={millis}, nonce=random, token={refresh_token}",
    "Content-Type": "application/json",
}

r = requests.post(url = ACCESS_TOKEN_URL, headers=headers) 
r_dict= json.loads(r.text)
access_token = r_dict['securityToken']
print("Access Token Obtained:")
print(access_token+"\n") 

## Upload Licence

In [ ]:
UPLOAD_LICENSE_URL = f'{YF_BASE_URL}/api/rpc/licence-management/upload-licence'

payload={}

with open("<licence_file_path>.lic", "rb") as license_file:

    files = [
        (
            "newLicence",
            (
                "<licence_file_name>.lic",
                license_file,
                "application/octet-stream",
            ),
        )
    ]

    millis = int(round(time.time() * 1000))
    headers = {
    'Accept': 'application/vnd.yellowfin.api-v1+json',
    'Authorization': f'YELLOWFIN ts={millis}, nonce=random, token={access_token}',
    }

    r = requests.request("POST", url=UPLOAD_LICENSE_URL, headers=headers, data=payload, files=files)
    assert r.status_code == 200, f"That didn't work, I got this instead: {r.text}"
    print("License uploaded sucessfully ✅")